# BFCL: Generate Planned Function Call Trajectories

In [ ]:
!pip install --quiet huggingface-hub litellm

In [1]:
import bfcl_runner
import litellm
import pandas as pd
import re
import logging
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict, Any

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [3]:
# Load a BFCL dataset
dataset_filename = "BFCL_v3_exec_simple.json"
df_raw = bfcl_runner.get_bfcl_dataset(dataset_filename)
# Prepare DataFrame
instructions = [item[0][0]['content'] for item in df_raw['question']]
tools_list = df_raw['function']
ground_truth_calls = [bfcl_runner.parse_ground_truth(gt) for gt in df_raw['ground_truth']]
bfcl_df = pd.DataFrame({
    'instruction': instructions,
    'tools': tools_list,
    'ground_truth_calls': ground_truth_calls,
})

In [4]:
# Convert raw functions to tool dicts
bfcl_df['tool_dicts'] = bfcl_df['tools'].apply(bfcl_runner.get_tool_dict_list)

In [7]:
# JSON parser for model output
def parse_json_plan(response_text: str) -> List[Dict[str, Any]]:
    import json, re, logging
    try:
        json_match = re.search(r'(\[.*\])', response_text, re.DOTALL)
        if json_match:
            plan = json.loads(json_match.group(1))
            if isinstance(plan, list):
                return [item for item in plan if isinstance(item, dict) and 'name' in item and 'arguments' in item]
        logging.warning('Failed to parse plan or wrong format')
        return []
    except Exception as e:
        logging.error(f'Error parsing JSON plan: {e}')
        return [{'error': str(e), 'raw_response': response_text}]

In [8]:
# Plan generation function
def generate_single_shot_plan(  user_query: str,  tools: List[Dict[str, Any]],  model: str,  api_base: str = None,  use_system_prompt: bool = False,) -> List[Dict[str, Any]]:
    if use_system_prompt:
        system_prompt = f"You are an agent that plans function calls.\nAvailable tools: {tools}\nRespond ONLY with a JSON array of {{'name':..., 'arguments':...}}."
        messages = [
            {'role':'system','content':system_prompt},
            {'role':'user','content':user_query},
        ]
    else:
        combined_prompt = (
    "Instructions: Plan function calls to answer user.\n"
    f"Tools: {tools}\n"
    f"User: {user_query}\n"
    "Respond ONLY with a JSON array of {{'name':..., 'arguments':...}}."
)
        messages = [{'role':'user','content':combined_prompt}]
    try:
        response = litellm.completion(model=model, api_base=api_base, messages=messages, temperature=0.0, max_tokens=1024,)
        raw = response.choices[0].message.content.strip()
        print('Model raw response:', raw)
        plan = parse_json_plan(raw)
        print('Parsed plan:', plan)
        return plan
    except Exception as e:
        print(f'Error generating plan: {e}')
        return [{'error':str(e)}]

In [11]:
AVAILABLE_MODELS = {
    "gemma-3-4b": "openai/google/gemma-3-4b-it",
    "llama-guard": "openai/meta-llama/Meta-Llama-Guard-2-8B", 
    "mistral-7b": "openai/mistralai/Mistral-7B-Instruct-v0.1"
}

# Select model by uncommenting one of these lines
MODEL_NAME = AVAILABLE_MODELS["gemma-3-4b"]
# MODEL_NAME = AVAILABLE_MODELS["llama-guard"]
# MODEL_NAME = AVAILABLE_MODELS["mistral-7b"]

In [12]:
model_configs = {
    "openai/mistralai/Mistral-7B-Instruct-v0.1": {"model": "openai/mistralai/Mistral-7B-Instruct-v0.1", "use_system_prompt": True, "api_base": "http://localhost:8000/v1"},
    "openai/google/gemma-3-4b-it": {"model": "openai/google/gemma-3-4b-it", "use_system_prompt": True, "api_base": "http://localhost:8000/v1"},
    "openai/meta-llama/Meta-Llama-Guard-2-8B": {"model": "openai/meta-llama/Meta-Llama-Guard-2-8B", "use_system_prompt": False, "api_base": "http://localhost:8000/v1"} 
}

In [13]:
model_config = model_configs[MODEL_NAME]

In [14]:
# Generate plans in parallel
bfcl_df['bfcl_planned_calls'] = None
def process_row(args):  idx, instr, tools = args;  return idx, generate_single_shot_plan(instr, tools, model, api_base, use_system_prompt)
from tqdm import tqdm
with ThreadPoolExecutor() as executor:
    args_list = [(i, row['instruction'], row['tool_dicts']) for i, row in bfcl_df.iterrows()]
    for idx, plan in tqdm(executor.map(process_row, args_list), total=len(args_list), desc='Processing BFCL'):
        bfcl_df.at[idx, 'bfcl_planned_calls'] = plan

17:17:10 - LiteLLM:INFO: utils.py:3075 - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
2025-04-18 17:17:10,897 - INFO - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
17:17:10 - LiteLLM:INFO: utils.py:3075 - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
2025-04-18 17:17:10,898 - INFO - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
17:17:10 - LiteLLM:INFO: utils.py:3075 - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
17:17:10 - LiteLLM:INFO: utils.py:3075 - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
2025-04-18 17:17:10,899 - INFO - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
17:17:10 - LiteLLM:INFO: utils.py:3075 - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
2025-04-18 17:17:10,903 - INFO - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
17:17:10 - LiteLLM:INFO: utils.py:3075 - 


Model raw response:Model raw response: ```json
[
  {
    "name": "calculate_density",
    "arguments": {
      "mass": 50.0,
      "volume": 10.0
    }
  }
]
```
Parsed plan: [{'name': 'calculate_density', 'arguments': {'mass': 50.0, 'volume': 10.0}}]
Model raw response: ```json
[
  {
    "name": "calculate_permutations",
    "arguments": {
      "n": 26,
      "k": 5
    }
  }
]
```
Parsed plan: [{'name': 'calculate_permutations', 'arguments': {'n': 26, 'k': 5}}]
 ```json
[
  {
    "name": "calculate_density",
    "arguments": {
      "mass": 120.0,
      "volume": 30.0
    }
  }
]
```
Parsed plan: [{'name': 'calculate_density', 'arguments': {'mass': 120.0, 'volume': 30.0}}]
Model raw response: ```json
[
  {
    "name": "calculate_permutations",
    "arguments": {
      "n": 30,
      "k": 7
    }
  }
]
```
Parsed plan: [{'name': 'calculate_permutations', 'arguments': {'n': 30, 'k': 7}}]
Model raw response: ```json
[
  {
    "name": "calculate_electrostatic_potential_energy",
    "arg

2025-04-18 17:17:15,749 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
17:17:15 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: openai/google/gemma-3-4b-it
17:17:15 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: google/gemma-3-4b-it
2025-04-18 17:17:15,749 - INFO - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
17:17:15 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
2025-04-18 17:17:15,750 - INFO - selected model name for cost calculation: openai/google/gemma-3-4b-it
17:17:15 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: google/gemma-3-4b-it
2025-04-18 17:17:15,751 - INFO - 
LiteLLM completion() model= google/gemma-3-4b-it; provider = openai
2025-04-18 17:17:15,752 - INFO - selected model name for cost calculation: openai/google/gemma-3-4b-it
17:17:15 - LiteLLM:INFO: cost_calculator.p

Model raw response: ```json
[
  {
    "name": "calculate_final_velocity",
    "arguments": {
      "initial_velocity": 0.0,
      "acceleration": 9.8,
      "time": 12.0
    }
  }
]
```
Parsed plan: [{'name': 'calculate_final_velocity', 'arguments': {'initial_velocity': 0.0, 'acceleration': 9.8, 'time': 12.0}}]
Model raw response: ```json
[
  {
    "name": "calculate_future_value",
    "arguments": {
      "present_value": 8000.0,
      "interest_rate": 0.04,
      "periods": 15
    }
  }
]
```
Parsed plan: [{'name': 'calculate_future_value', 'arguments': {'present_value': 8000.0, 'interest_rate': 0.04, 'periods': 15}}]
Model raw response: ```json
[
  {
    "name": "calculate_future_value",
    "arguments": {
      "present_value": 5000.0,
      "interest_rate": 0.05,
      "periods": 10
    }
  }
]
```
Parsed plan: [{'name': 'calculate_future_value', 'arguments': {'present_value': 5000.0, 'interest_rate': 0.05, 'periods': 10}}]


2025-04-18 17:17:16,319 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
17:17:16 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
2025-04-18 17:17:16,321 - INFO - Wrapper: Completed Call, calling success_handler
17:17:16 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: openai/google/gemma-3-4b-it
2025-04-18 17:17:16,322 - INFO - selected model name for cost calculation: openai/google/gemma-3-4b-it
17:17:16 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: openai/google/gemma-3-4b-it
2025-04-18 17:17:16,322 - INFO - selected model name for cost calculation: openai/google/gemma-3-4b-it
17:17:16 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: google/gemma-3-4b-it
2025-04-18 17:17:16,322 - INFO - selected model name for cost calculation: google/gemma-3-4b-it
17:17:16 - LiteLLM:INFO: cost_calculator.py:622 - selected mo

...

Model raw response: ```json
[
  {
    "name": "polygon_area",
    "arguments": {
      "vertices": [
        [
          1,
          2
        ],
        [
          3,
          4
        ],
        [
          1,
          3
        ]
      ]
    }
  }
]
```
Parsed plan: [{'name': 'polygon_area', 'arguments': {'vertices': [[1, 2], [3, 4], [1, 3]]}}]
Model raw response: ```json
[
  {
    "name": "book_room",
    "arguments": {
      "room_type": "deluxe",
      "price": 1000.0,
      "check_in_date": "08-11-2024",
      "check_out_date": "08-15-2024",
      "customer_id": "123"
    }
  }
]
```
Parsed plan: [{'name': 'book_room', 'arguments': {'room_type': 'deluxe', 'price': 1000.0, 'check_in_date': '08-11-2024', 'check_out_date': '08-15-2024', 'customer_id': '123'}}]
Model raw response: ```json
[
  {
    "name": "polygon_area",
    "arguments": {
      "vertices": [
        [
          1,
          2
        ],
        [
          3,
          4
        ],
        [
          1,
    

17:17:20 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
2025-04-18 17:17:20,431 - INFO - Wrapper: Completed Call, calling success_handler
17:17:20 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: openai/google/gemma-3-4b-it
2025-04-18 17:17:20,432 - INFO - selected model name for cost calculation: openai/google/gemma-3-4b-it
17:17:20 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: google/gemma-3-4b-it
2025-04-18 17:17:20,433 - INFO - selected model name for cost calculation: google/gemma-3-4b-it
17:17:20 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: openai/google/gemma-3-4b-it
2025-04-18 17:17:20,435 - INFO - selected model name for cost calculation: openai/google/gemma-3-4b-it
17:17:20 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: google/gemma-3-4b-it
2025-04-18 17:17:20,437 - INFO - selected model name for cos

Model raw response: ```json
[
  {
    "name": "book_room",
    "arguments": {
      "room_type": "king",
      "price": 10000.0,
      "check_in_date": "12-11-2023",
      "check_out_date": "08-15-2024",
      "customer_id": "123",
      "discount_code": null
    }
  }
]
```
Parsed plan: [{'name': 'book_room', 'arguments': {'room_type': 'king', 'price': 10000.0, 'check_in_date': '12-11-2023', 'check_out_date': '08-15-2024', 'customer_id': '123', 'discount_code': None}}]


2025-04-18 17:17:20,683 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
17:17:20 - LiteLLM:INFO: utils.py:1177 - Wrapper: Completed Call, calling success_handler
2025-04-18 17:17:20,684 - INFO - Wrapper: Completed Call, calling success_handler
17:17:20 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: openai/google/gemma-3-4b-it
2025-04-18 17:17:20,685 - INFO - selected model name for cost calculation: openai/google/gemma-3-4b-it
17:17:20 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: openai/google/gemma-3-4b-it
17:17:20 - LiteLLM:INFO: cost_calculator.py:622 - selected model name for cost calculation: google/gemma-3-4b-it
2025-04-18 17:17:20,685 - INFO - selected model name for cost calculation: openai/google/gemma-3-4b-it
2025-04-18 17:17:20,686 - INFO - selected model name for cost calculation: google/gemma-3-4b-it
17:17:20 - LiteLLM:INFO: cost_calculator.py:622 - selected mo

Model raw response: ```json
[
  {
    "name": "order_food",
    "arguments": {
      "item": [
        "burger"
      ],
      "quantity": [
        10
      ],
      "price": [
        5.0
      ]
    }
  },
  {
    "name": "order_food",
    "arguments": {
      "item": [
        "ice cream"
      ],
      "quantity": [
        7
      ],
      "price": [
        2.0
      ]
    }
  }
]
```
Parsed plan: [{'name': 'order_food', 'arguments': {'item': ['burger'], 'quantity': [10], 'price': [5.0]}}, {'name': 'order_food', 'arguments': {'item': ['ice cream'], 'quantity': [7], 'price': [2.0]}}]
Model raw response: ```json
[
  {
    "name": "order_food",
    "arguments": {
      "item": [
        "dumplings"
      ],
      "quantity": [
        101
      ],
      "price": [
        0.1
      ]
    }
  },
  {
    "name": "order_food",
    "arguments": {
      "item": [
        "rice bowl"
      ],
      "quantity": [
        20
      ],
      "price": [
        10.0
      ]
    }
  }
]
```
Pa

In [15]:
# Save results
bfcl_df = bfcl_df.drop(columns=['tools','tool_dicts'])
bfcl_df.to_parquet('bfcl_eval_results.parquet', engine='fastparquet')